In [1]:
%%writefile clue.py

from sys import exit
from printouts import *
from errors import *
from board import *
from detective import *


class Engine():
    """This class moves the clue detective program from one state to another.
       
       Attributes:
       - board = current instance of the Board class 
       - notebook = current instance of the Notebook class 
       - detective = current instance of the Detective class
       
       Functions:
       - setup(): Sets up the particular instance of Board, Notebook, and Detective classes. 
       - take_turn(): Takes the player whose turn it is as an argument.  
                      Manages all that needs to happen during a turn: 
                      creates a current instance of Turn, 
                      calls turn_input function, 
                      passes current turn information to the detective, 
                      calls deductions function and prints out current notes, 
                      and determines whose turn is next.
       - turn_input(): Takes the player whose turn it is and an instance of Turn as an argument.  
                       Prompts user to input information about the turn: what suspect, room, 
                       and weapon was suggested, who disproved it (if any), what card was revealed (if any).  
                       Stores information in the current turn.
       - deductions(): Checks what is stored in the solution.  
                       If no solution or partial solution, go back to taking turns.  
                       If full solution is found, print and exit the program.
    """

    def __init__(self, board=None, notebook=None, detective=None):
        self.board = board
        self.notebook = notebook
        self.detective = detective
        self.setup()
    
    
    def setup(self):
        welcome()
        instructions()
        print()
        print("Let's start by setting up your board!")
        print()
        
        q1 = "Which suspect are you? "
        q2 = "\nPress ENTER to accept or type N to re-do: "
        
        # prompt user to enter their name
        while not is_valid_player(me_input := input(q1)):
            print("Please enter a valid Suspect.")     
        me_decoded = Board.identify(me_input)
        print(f"Hello, {me_decoded.name}!")
        print()
        
        # prompt user to enter the players in their turn order
        print("Enter ALL the players one by one in the order of their turn.")
        print("When you are done, enter #")
        accept = 'N'
        while accept == 'N':
            players_decoded, suspects_decoded = collect_players_and_suspects_list()
            if len(players_decoded) == 1:
                print("Not enough players. Try again.")
                continue
            elif len(players_decoded) > 6:
                print("Too many players. Try again.")
                continue            
            print("The players in order are:", end=" ")
            for player in players_decoded:
                print(player, end="  ")
            accept = input(q2).upper()
            if accept == 'Q':
                exit("\nExiting program. Thanks for using Clue Detective!\n")
        print()
                
        # prompt user to enter other suspects that aren't players
        # if number of players is not 6
        number = 6 - len(players_decoded)
        if number > 0:
            print("Enter the rest of the suspects that are not players")
            print("When you are done, enter #")
            accept = 'N'
            more_suspects = []
            while accept == 'N':
                more_suspects = collect_cards()
                if len(more_suspects) != number:
                    print(f"You need to enter {number} suspects. Try again.")
                    continue          
                print("The players in order are:", end=" ")
                for suspect in more_suspects:
                     print(suspect, end="  ")
                accept = input(q2).upper()
                if accept == 'Q':
                    exit("\nExiting program. Thanks for using Clue Detective!\n")
        
            # add suspects that aren't players to suspects list
            for suspect in more_suspects:
                suspects_decoded.append(suspect)
        
        # create an instance of Board
        b = Board(me_decoded, players_decoded, suspects_decoded)
        self.board = b
        print()
        
        # figure out the number of cards each player has
        if len(players_decoded) == 4 or len(players_decoded) == 5:
            print("Enter the two players with less cards one by one.")
            print("When you are done, enter #")
            accept2 = 'N'
            while accept2 == 'N':
                different_decoded = collect_players_list()
                if len(different_decoded) != 2:
                    print("Please enter two valid players.")
                    continue           
                print("The two players are:", end=" ")
                for player in different_decoded:
                    print(player.name, end="  ")
                accept2 = input(q2).upper()
                if accept2 == 'Q':
                    exit("\nExiting program. Thanks for using Clue Detective!\n")
            b.num_card_calc(different_decoded)
            print()
        else:
            b.num_card_calc()
        
        # prompt user to enter the cards in their hand
        print("Enter all the cards in your hand one by one.")
        print("When you are done, enter #")
        accept3 = 'N'
        while accept3 == 'N':
            hand_decoded = collect_cards()
            if len(hand_decoded) != self.board.me.num_cards:
                print(f"You need to enter {self.board.me.num_cards} cards. Try again.")
                continue           
            print("Your hand is:", end=" ")
            for card in hand_decoded:
                print(card, end="  ")
            accept3 = input(q2).upper()
            if accept3 == 'Q':
                exit("\nExiting program. Thanks for using Clue Detective!\n")
        b.me.hand = hand_decoded
        print()
        
        # create an instance of Notebook and Detective
        n = Notebook(self.board)
        self.notebook = n
        d = Detective(self.board, self.notebook)
        self.detective = d
        
        print("Your board is now set up!  We're ready to play Clue!")
        print()
        self.notebook.printout()
        
        self.take_turn(players_decoded[0])
        
     
    def take_turn(self, player):
                
        current_turn = self.board.log_turn(player)
        i = self.board.history.index(current_turn)
        print()
        print("~"*22+f" START TURN #{i+1:<2d} " +"~"*21)
        print()
        
        # use turn_input function to prompt user for turn information
        self.turn_input(player, current_turn)
        
        print()
        print("~"*23+f" END TURN #{i+1:<2d} "+"~"*22)
        print()
        
        self.detective.take_notes(current_turn)
        self.deductions()
        
        next_player = self.board.next_player(player)
        self.take_turn(next_player)

   
    def turn_input(self, player, turn):
        if player == self.board.me:
            print("It's YOUR turn!")
        else:
            print(f"It is {player}'s turn.")
        print()
        
        q1 = "Which ROOM did {} enter? \nTo skip this turn, type X: ".format(player)
        q2 = "Which SUSPECT did {} guess? ".format(player)
        q3 = "Which WEAPON did {} guess? ".format(player)
        q4 = "Which player disproved the suggestion? \nIf no one disproved it, type X: "
        q5 = "What card was revealed? "
        q6 = "Press ENTER to accept or type N to re-do: "
        
        # prompt user to input all the information for the turn
        # handles typos and allows user to review entry
        # allows user to quit at any time
        accept = 'N'
        while accept == 'N':
            
            # prompt user to enter room or skip the turn
            while not is_valid(room_input := input(q1), 'Room', skip=True):
                print("Please enter a valid Room.")
            if room_input.upper() == 'X':
                print()
                print("~"*25+" END TURN "+"~"*25)
                next_player = self.board.next_player(player)
                self.take_turn(next_player)
            room_decoded = Board.translate(room_input)
            print()
        
            # prompt user to enter suspect
            while not is_valid(suspect_input := input(q2), 'Suspect'):
                print("Please enter a valid Suspect.")
            suspect_decoded = Board.translate(suspect_input)
            print()
        
            # prompt user to enter weapon
            while not is_valid(weapon_input := input(q3), 'Weapon'):
                print("Please enter a valid Weapon.")
            weapon_decoded = Board.translate(weapon_input)
            print()
            
            # store suggestion in current turn
            suggestion = (suspect_decoded, weapon_decoded, room_decoded)
            turn.suggestion = suggestion
        
            # prompt user to enter disprover, if any
            while not is_valid(disprover_input := input(q4), 'Suspect', skip=True):
                print("Please enter a valid Suspect.")
            if disprover_input.upper() != 'X':
                disprover_decoded = Board.translate(disprover_input)
                turn.disprover = disprover_decoded
            print()
        
            # prompt user to enter revealed card, if any
            if player == self.board.me and disprover_input != 'X':
                while not is_valid(revealed_input := input(q5)):
                    print("Please enter a valid card.")
                revealed_decoded = Board.translate(revealed_input)
                turn.revealed = revealed_decoded
                print()
                
            # print confirmation of entry for user to review
            if player == self.board.me:
                print(f"You suggested {suspect_decoded} with the {weapon_decoded} in the {room_decoded}.")
                if disprover_input.upper() == 'X':
                    print("No player disproved.")
                else:
                    print(f"{disprover_decoded} disproved and revealed {revealed_decoded}.")
            else:
                print(f"{player} suggested {suspect_decoded} with the {weapon_decoded} in the {room_decoded}.")
                if disprover_input.upper() == 'X':
                    print("No player disproved.")
                else:
                    print(f"{disprover_decoded} disproved.")
            
            # allows user to accept or reject entry and try again
            accept = input(q6).upper()
            print()
            if accept == 'Q':
                exit("\nExiting program. Thanks for using Clue Detective!\n")
 

    def deductions(self):
        print()
        print("*"*24+" DEDUCTIONS "+"*"*24)
        print()
        notes_legend()
        print()
              
        if self.board.solution.is_solved():
            suspect = self.board.solution.suspect
            weapon = self.board.solution.weapon
            room = self.board.solution.room
            solution_banner(suspect, weapon, room)
            print()
            exit("Exiting program. Thanks for using Clue Detective!\n")
        
        elif self.board.solution.not_solved():
            print("No solutions found yet.")
            print()
            self.notebook.printout()
        
        elif self.board.solution.partially_solved():
            print("Partial solution found.")
            if self.board.solution.suspect:
                print("SUSPECT = ", self.board.solution.suspect)
            if self.board.solution.weapon:
                print("WEAPON = ", self.board.solution.weapon)
            if self.board.solution.room:
                print("ROOM = ", self.board.solution.room)
            print()
            self.notebook.printout()

if __name__ == "__main__":
    start = Engine()

Overwriting clue.py


In [2]:
%%writefile board.py

class Player():
    """This is a container class used to store all the information about the players of the game.
       
       Attributes:
       - name = name of the player
       - num_cards = number of cards in the player’s hand
       - hand = list of cards in the player’s hand
    """
    
    def __init__(self, name, num_cards=None, hand=[]):
        self.name = name
        self.num_cards = num_cards
        self.hand = hand    
    def __eq__(self, other):
        return self.name == other.name 
    def __repr__(self):
        return self.name
    

class Solution():
    """This is a container class used to store all the solutions found by the detective.

       Attributes:
       - suspect = guess for suspect solution
       - weapon = guess for weapon solution
       - room = guess for room solution
    """
    
    def __init__(self, suspect=None, weapon=None, room=None):
        self.name = 'Solution'
        self.suspect = suspect
        self.weapon = weapon
        self.room = room
        
    def is_solved(self):
        if self.suspect and self.weapon and self.room:
            return True
        else:
            return False
    
    def not_solved(self):
        if not self.suspect and not self.weapon and not self.room:
            return True
        else:
            return False
    
    def partially_solved(self):
        if self.suspect or self.weapon or self.room:
            return True
        else:
            return False

        
class Card():
    """This is class represents the cards in the game of Clue.

       Attributes:
       - name = name of the card

       Subclasses:
       - Suspect: has the attribute type ‘Suspect’
       - Weapon: has the attribute type ‘Weapon’
       - Room: has the attribute type ‘Room’
    """
    
    def __init__(self, name):
        self.name = name
    def __eq__(self, other):
        self.name == other.name    
    def __repr__(self):
        return self.name

class Suspect(Card):
    
    def __init__(self, name):
        super().__init__(name)
        self.type = 'Suspect'

class Weapon(Card):
    
    def __init__(self, name):
        super().__init__(name)
        self.type = 'Weapon'
        
class Room(Card):
    
    def __init__(self, name):
        super().__init__(name)
        self.type = 'Room'


class Turn():
    """This is a container class used to store all the information for one turn.

       Attributes:
       suggester = player whose turn it is
       suggestion = tuple containing the suspect, room, and weapon that was suggested
       disprover = player who disproved the suggestion
       revealed = card that was revealed that turn
    """
    
    def __init__(self, suggester):
        self.suggester = suggester
        suggestion = ()
        self.suggestion = suggestion
        disprover = None
        self.disprover = disprover
        revealed = None
        self.revealed = revealed


class Board():
    """This is a container class used to store all the information about the current game of Clue being played by the user.

       Attributes:
       - *start_cards = list that stores all the starting cards in the Clue game (class attribute)
       - *input_decoder = dictionary for decoding input codes for the cards (class attribute)
       - *player_decoder = dictionary for decoding input codes for the players (class attribute)
       - me: which player the user is
       - players = list of the players in the order of their turn
       - solution = particular instance of the Solution class used in the game
       - cards = cards used in the game
       - history = list of all the turns made in the game

       Functions:
       - translate(): Takes an input code as argument and returns the card it decodes to
       - identify(): Takes an input code as argument and returns the player it decodes to
       - num_card_calc(): Calculates the number of cards each player should have for 2, 3, and 6 player game.  
                          For 4 or 5 player game, takes in which players have less cards 
                          and assigns the number of cards accordingly.  
                          Stores the number of cards for each player in the num_cards attribute.
       - next_player(): Takes in the player whose turn it is as argument 
                        and returns the player whose turn is next.
       - log_turn(): Takes in information about the turn 
                     and returns an instance of the turn with the information recorded in it.
    """
    
    start_cards = [Weapon('Candlestick'), Weapon('Dagger'), Weapon('Lead Pipe'),
             Weapon('Revolver'), Weapon('Rope'), Weapon('Wrench'),
             Room('Ballroom'), Room('Billiard Room'), Room('Conservatory'),
             Room('Dining Room'), Room('Hall'), Room('Kitchen'),
             Room('Library'), Room('Lounge'), Room('Study')]
    
    input_decoder = {'MU': Suspect('Mustard'), 'OR': Suspect('Orchid'), 'SC': Suspect('Scarlett'), 
             'GR': Suspect('Green'), 'PE': Suspect('Peacock'), 'PL': Suspect('Plum'), 
             'WH': Suspect('White'), 'BR': Suspect('Brunette'), 'AZ': Suspect('Azure'),
             'RU': Suspect('Rusty'), 'SH': Suspect('Sherlock'), 'WA': Suspect('Watson'),
             'MO': Suspect('Moriarty'), 'AD': Suspect('Adler'), 'ME': Suspect('Meadow-Brook'), 
             'RS': Suspect('Rose'), 'PH': Suspect('Peach'), 'GY': Suspect('Gray'),
             'CA': Weapon('Candlestick'), 'DA': Weapon('Dagger'), 'LE': Weapon('Lead Pipe'),
             'RE': Weapon('Revolver'), 'RO': Weapon('Rope'), 'WR': Weapon('Wrench'),
             'BA': Room('Ballroom'), 'BI': Room('Billiard Room'), 'CO': Room('Conservatory'),
             'DI': Room('Dining Room'), 'HA': Room('Hall'), 'KI': Room('Kitchen'),
             'LI': Room('Library'), 'LO': Room('Lounge'), 'ST': Room('Study')}
    
    player_decoder = {'MU': Player('Mustard'), 'OR': Player('Orchid'), 'SC': Player('Scarlett'), 
             'GR': Player('Green'), 'PE': Player('Peacock'), 'PL': Player('Plum'),
             'WH': Player('White'), 'BR': Player('Brunette'), 'AZ': Player('Azure'),
             'RU': Player('Rusty'), 'SH': Player('Sherlock'), 'WA': Player('Watson'),
             'MO': Player('Moriarty'), 'AD': Player('Adler'), 'ME': Player('Meadow-Brook'),
             'RS': Player('Rose'), 'PH': Player('Peach'), 'GY': Player('Gray')}

    
    def __init__(self, me, players, suspects):
        
        self.me = me
        self.players = players
        self.num_players = len(players)
        self.solution = Solution()
        self.cards = Board.start_cards
        for suspect in suspects:
            self.cards.append(suspect)
        history = []
        self.history = history
    
    def translate(user_input):
        i = user_input.upper()
        return Board.input_decoder[i]
    
    def identify(user_input):
        i = user_input.upper()
        return Board.player_decoder[i]
    
    def num_card_calc(self, different = []):
        even = [2, 3, 6]
        if self.num_players in even:
            for player in self.players:
                player.num_cards = int(18 / self.num_players)
        elif self.num_players == 4:
            for player in self.players:
                if player in different:
                    player.num_cards = int(4)
                else: 
                    player.num_cards = int(5)
        elif self.num_players == 5:
            for player in self.players:
                if player in different:
                    player.num_cards = int(3)
                else:
                    player.num_cards = int(4)
              
    def next_player(self, player):
        i = self.players.index(player)
        n = self.num_players
        return self.players[(i+1)%n]
    
    def log_turn(self, suggester, suggestion=(), disprover=None, revealed=None):
        turn = Turn(suggester)
        if suggestion:
            turn.suggestion = suggestion
        if disprover:
            turn.disprover = disprover
        if revealed:
            turn.revealed = revealed
        self.history.append(turn)
        return turn

Overwriting board.py


In [3]:
%%writefile detective.py

from board import *

class Note():
    """This is class represents the types of notes stored in a notebook.

       Attributes:
       - type = what type of information the note contains

       Subclasses:
       - Yes: marks the cards that are known to be in another player’s hand
       - No: marks the cards that are known to be absent from another player’s hand
       - Maybe: marks the unknown cards that a player secretly revealed in a turn 
       - Mine: marks the cards that are in the user’s hand
    """
    
    def __init__(self):
        pass
    def __eq__(self, other):
        return self.type == other.type
    
class Yes(Note):
    def __init__(self):
        self.type = 'yes'
    def __repr__(self):
        return '\u2713'

class No(Note):
    def __init__(self):
        self.type = 'no'
    def __repr__(self):
        return 'X'

class Maybe(Note):
    def __init__(self):
        self.type = 'maybe'
        self.notes = []
    def __repr__(self):
        if len(self.notes) == 0:
            return ' '
        elif len(self.notes) == 1:
            return '?'
        elif len(self.notes) > 1:
            return '!'

class Mine(Note):
    def __init__(self):
        self.type = 'mine'
    def __repr__(self):
        return '\u2731'


class Notebook():
    """This is a container class used to store all the notes taken by the detective during 
       the current game of Clue being played by the user.

       Attributes:
       - board:  current instance of Board
       - contents:  dictionary that stores all the notes indexed by card and player

       Functions:
       - mark_mine():   Marks the appropriate notes for all the cards in the user’s hand.
       - mark_no():  Marks a card as absent in a player’s hand.
       - mark_yes():  Marks the card as present in a player’s hand and absent in everyone else’s.
       - mark_maybe():  Marks the unknown cards that a player secretly revealed that turn.
       - rows():  Takes in a card type as argument and returns all the rows in the notebook 
                  containing cards of that type.
       - printout():  Prints out the current contents of the notebook.
    """
    
    def __init__(self, board):
        self.board = board
        self.contents = {(card.type, card.name, player.name): Maybe() 
                         for card in board.cards 
                         for player in board.players}
        self.mark_mine()

    def mark_mine(self):
        # mark all the cards in the user's hand
        keys = [(card.type, card.name, self.board.me.name) for card in self.board.me.hand]
        for key in keys:
            self.contents[key] = Mine()
        
        # cross out all cards in the user's column that is absent from their hand
        x_keys = [key for key in self.contents.keys() 
                  if key[2] == self.board.me.name and self.contents[key] != Mine()]
        for key in x_keys:
            self.mark_no(key)
        
        # cross out the cards in the user's hand for all the other players
        my_hand = [card.name for card in self.board.me.hand]
        x_keys2 = [key for key in self.contents 
                   if key[2] != self.board.me.name and key[1] in my_hand]
        for key in x_keys2:
            self.mark_no(key)
            
    def mark_no(self, key):
        
        # only proceed if card is orginally marked as maybe
        not_maybe = ['yes', 'no', 'mine']
        if self.contents[key].type in not_maybe:
            return
        else: 
            self.contents[key] = No()
       
    def mark_yes(self, key):
        
        # only proceed if card is orginally marked as maybe
        not_maybe = ['yes', 'no', 'mine']
        if self.contents[key].type in not_maybe:
            print("Warning: something is wrong with the entry; not recorded.")
            return
        
        # if card was marked as a maybe for a turn
        # remove those maybe marks for other cards that remain
        # this is needed for a deduction step later
        if len(self.contents[key].notes) > 0:
            player_maybe = [new_key for new_key in self.contents 
                            if new_key[2] == key[2] 
                            and self.contents[new_key].type == 'maybe']
            for new_key in player_maybe:
                if len(self.contents[new_key].notes) > 0:
                    for i in self.contents[new_key].notes:
                        if i in self.contents[key].notes:
                            self.contents[new_key].notes.remove(i)
        
        # mark the card with check
        self.contents[key] = Yes()
        
        # cross out the card for all the other players
        x_keys = [x_key for x_key in self.contents
                 if x_key[2] != key[2] and x_key[1] == key[1]]
        for x in x_keys:
            self.mark_no(x)
    
    def mark_maybe(self, turn):
        i = self.board.history.index(turn)
        for card in turn.suggestion:
            key = (card.type, card.name, turn.disprover.name)
            if self.contents[key].type == 'maybe':
                self.contents[key].notes.append(i)
           
    def rows(self, card_type):
        rows = {}
        for key in self.contents:
            if key[0] == card_type:
                if key[1] not in rows:
                    rows[key[1]] = []
                rows[key[1]].append(self.contents[key])
        return rows  
    
    def printout(self):
        
        border = '-'*(16 + 4*self.board.num_players)
        print(f"{' ':17s}", end = ' ')
        for player in self.board.players:
            if player.name == 'Rose':
                print('RS', end=" ")
            elif player.name == 'Peach':
                print('PH', end=" ")
            elif player.name == 'Gray':
                print('GY', end=" ")
            else:
                print(f"{player.name[:2].upper():^3s}", end=" ")
        print('\n' + border)
        
        card_types = ['Suspect', 'Weapon', 'Room']
        for t in card_types:
            cards = self.rows(t)
            print(t.upper() + ":" )
            for card in cards.keys():
                print(f"{card:13s}", '-|-', end = ' ')
                for player in self.board.players:
                    symbol = str(self.contents[(t, card, player.name)])
                    print(f"{symbol:^3s}", end = ' ')
                print()
            print('\n', border)
        print()


class Detective():
    """This is class takes notes on information learned in each turn 
       and deduced new information from previous notes. 

       Attributes:
       - board: current instance of Board
       - notebook = current instance of Notebook

       Functions:
       - take notes(): Takes in a turn as argument. If there is no disprover, crosses out 
                       the cards suggested for all the players except the suggester.  
                       If there is a disprover and a card is revealed, checks off the card.  
                       If there is a disprover, but the card is revealed secretly, marks all the maybes.
       - deduce_count(): Check to see if all the player’s cards are known.  
                         If they are, cross off all the other cards for that player.
       - deduce_maybe(): Check a player’s column for any cards from previous turns that are 
                         now known to be in that player’s hand by process of elimination.
       - deduce_solution(): Takes a card type as an argument, checks for a solution for that type, 
                            and returns the solution, if found.  
       - find_solution(): Uses the deduce_solution function to find any solutions and stores it in 
                          the current instance of Solution.
    """

    def __init__(self, board, notebook):
        self.board = board
        self.notebook = notebook

        
    def take_notes(self, turn):
        suggestion = turn.suggestion
        revealed = turn.revealed
        disprover = turn.disprover
        suggester = turn.suggester

        if not disprover:
            x_keys = [(card.type, card.name, player.name) 
                      for card in suggestion 
                      for player in self.board.players if player != suggester]
            for key in x_keys:
                self.notebook.mark_no(key)

        elif disprover != self.board.me:

            # check off the revealed card if any
            # or else mark secretly revealed unknown cards as maybe
            if revealed:
                key = (revealed.type, revealed.name, disprover.name)
                self.notebook.mark_yes(key)
                x_keys = [key for key in self.notebook.contents 
                          if key[1] == revealed.name 
                          and key[2] != disprover.name 
                          and key[2] != self.board.me.name]
                for key in x_keys:
                    self.notebook.mark_no(key)
            else: 
                self.notebook.mark_maybe(turn)

            # if players could not disprove before the disprover
            # mark the cards suggested as absent in all those players' hands
            x_players = []
            next_player = self.board.next_player(suggester)
            while next_player.name != disprover.name:
                x_players.append(next_player)
                current_player = next_player
                next_player = self.board.next_player(current_player)

            x_keys = [(card.type, card.name, player.name)
                      for card in suggestion 
                      for player in x_players if player != self.board.me]
            for key in x_keys:
                self.notebook.mark_no(key)

        self.deduce_count()
        self.deduce_maybe()
        self.deduce_count()
        self.deduce_maybe()
        self.deduce_count()
        
        self.find_solution()
                       
    
    def deduce_count(self):
        dictionary = self.notebook.contents
        for player in self.board.players:
            cards = [key for key in dictionary 
                     if key[2] == player.name 
                     and key[2] != self.board.me.name
                     and dictionary[key] != Yes()]
            yes_cards = 21 - len(cards)
            if player.num_cards == yes_cards:
                for key in cards:
                    self.notebook.mark_no(key)

                    
    def deduce_maybe(self):
        dictionary = self.notebook.contents
        for player in self.board.players:
            maybe_cards = [key for key in dictionary 
                           if key[2] == player.name and dictionary[key].type == 'maybe']
            if len(maybe_cards) > 0:
                maybe_short = maybe_cards.copy()
                values = []
                # look if the player has any maybes marked from previous turns
                for key in maybe_cards:
                    if len(dictionary[key].notes) == 0:
                        maybe_short.remove(key)
                    if len(dictionary[key].notes) > 0:
                        values.extend(dictionary[key].notes)
                # look to see if any maybes from previous turns have been isolated
                # if so, we know by process of elimination
                # that the player definitely has that card
                # only works if the safeguard in mark_yes() in notebook works
                if len(values) > 0:
                    value_count = [x for x in set(values) if values.count(x) == 1]
                    if len(value_count) > 0:
                        for key in maybe_short:
                            for i in dictionary[key].notes:
                                if i in value_count:
                                    self.notebook.mark_yes(key)
    
                                  
    def deduce_solution(self, card_type):
        check = [key for key in self.notebook.rows(card_type)] 
        for key, row in self.notebook.rows(card_type).items():
            count = 0
            # checks for the solution by row
            for entry in row:
                if entry == No():
                    count += 1
                # checks for the solution by column
                elif entry == Yes() or entry == Mine():
                    check.remove(key)
            if count == self.board.num_players:
                return key
        if len(check) == 1:
            return check[0]
                                
    
    def find_solution(self):
        solution = self.board.solution
        if solution.suspect == None:
            solution.suspect = self.deduce_solution('Suspect')
        if solution.weapon == None:
            solution.weapon = self.deduce_solution('Weapon')
        if solution.room == None:
            solution.room = self.deduce_solution('Room')

Overwriting detective.py


In [4]:
%%writefile printouts.py
"""This module contains helper functions for printing out decoder keys, legends, and banners."""

from board import *

def welcome():
    print("*"+"-"*58+"*")
    print("|"+" "*24+"WELCOME TO"+" "*24+"|")
    print("|"+" "*22+"CLUE DETECTIVE!"+" "*21+"|")
    print("*"+"-"*58+"*")

def instructions():
    print("*"+" "*58+"*")
    print("*"+" "*23+"HOW TO USE:"+" "*24+"*")
    print("*"+" "*4+"Use the given input codes to enter your responses"+" "*5+"*")
    print("*"+" "*2+"HINT: Most codes are the first two letters of the word"+" "*2+"*")
    print("*"+" "*9+"To quit the program at any time, enter Q"+" "*9+"*")
    print("*"+" "*58+"*")
    suspect_decoder()
    weapon_decoder()
    room_decoder()
    print()

def suspect_decoder():
    key = [key for key in Board.input_decoder if Board.input_decoder[key].type == 'Suspect']
    value = [value.name for value in Board.input_decoder.values() if value.type == 'Suspect']
    print("-"*22+" SUSPECT CODES "+"-"*23)
    print(" "*8+f"{key[0]} = {value[0]:8s}  {key[1]} = {value[1]:8s}  {key[2]} = {value[2]:7s}")
    print(" "*8+f"{key[3]} = {value[3]:8s}  {key[4]} = {value[4]:8s}  {key[5]} = {value[5]:7s}")
    print(" "*8+f"{key[6]} = {value[6]:8s}  {key[7]} = {value[7]:8s}  {key[8]} = {value[8]:7s}")
    print(" "*8+f"{key[9]} = {value[9]:8s}  {key[10]} = {value[10]:8s}  {key[11]} = {value[11]:7s}")
    print(" "*8+f"{key[12]} = {value[12]:8s}  {key[13]} = {value[13]:8s}  {key[14]} = {value[14]:7s}")
    print(" "*8+f"{key[15]} = {value[15]:8s}  {key[16]} = {value[16]:8s}  {key[17]} = {value[17]:7s}")
    print("-"*60)
    
def weapon_decoder():
    key = [key for key in Board.input_decoder if Board.input_decoder[key].type == 'Weapon']
    value = [value.name for value in Board.input_decoder.values() if value.type == 'Weapon']
    print("-"*23+" WEAPON CODES "+"-"*23)
    print(" "*5+f"{key[0]} = {value[0]:12s}  {key[1]} = {value[1]:8s}  {key[2]} = {value[2]:8s}"+" "*5)
    print(" "*5+f"{key[3]} = {value[3]:12s}  {key[4]} = {value[4]:8s}  {key[5]} = {value[5]:8s}"+" "*5)
    print("-"*60)

def room_decoder():
    key = [key for key in Board.input_decoder if Board.input_decoder[key].type == 'Room']
    value = [value.name for value in Board.input_decoder.values() if value.type == 'Room']
    print("-"*24+" ROOM CODES "+"-"*24)
    print(f" {key[0]} = {value[0]:12s}  {key[1]} = {value[1]:14s}  {key[2]} = {value[2]:8s}")
    print(f" {key[3]} = {value[3]:12s}  {key[4]} = {value[4]:14s}  {key[5]} = {value[5]:8s}")
    print(f" {key[6]} = {value[6]:12s}  {key[7]} = {value[7]:14s}  {key[8]} = {value[8]:8s}")
    print("-"*60)

def notes_legend():
    print("-"*23+" NOTES LEGEND "+"-"*23)
    print(" "*2+"\u2731 = In your hand", end = "   ")
    print("\u2713 = Has card", end = "   ")
    print("X = Does not have card"+" "*2)
    print(" "*10+"? = Might have card", end = "   ")
    print("! = Likely has card"+" "*9)
    print("-"*60)

def solution_banner(suspect, weapon, room):
    print("*"+"-"*58+"*")
    print("|"+" "*20+"SOLUTION FOUND!!!!"+" "*20+"|")
    print("|"+" "*20+f"It was {suspect:8s}"+" "*23+"|")
    print("|"+" "*20+f"with the {weapon:12s}"+" "*17+"|")
    print("|"+" "*20+f"in the {room:14s}"+" "*17+"|")
    print("*"+"-"*58+"*")

Overwriting printouts.py


In [5]:
%%writefile errors.py
"""This module contains helper functions for detecting errors in the user input from prompts."""

from board import *
from sys import exit

def is_valid_player(user_input):
    """This function determines if the user input is a valid player.
       If input is 'Q', exits program.
       
       Args: user input
       Returns: True or False
    """
    
    i = user_input.upper()
    if i in Board.player_decoder:
        return True
    elif i == 'Q':
        exit("\nExiting program. Thanks for using Clue Detective!\n")
    else:
        return False

def is_valid(user_input, card_type=None, skip=False):
    """This function determines if the user input is a valid card.
       If skip = True, also allows 'X' as a valid input.
       If input is 'Q', exits program.
       
       Args: user input
       Returns: True or False
    """
    
    i = user_input.upper()
    if i == 'Q':
        exit("\nExiting program. Thanks for using Clue Detective!\n")
    if skip:
        if i == 'X':
            return True
    if card_type:
        key_list = [key for key in Board.input_decoder 
                    if Board.input_decoder[key].type == card_type]
        if i in key_list:
            return True
    elif not card_type:
        if i in Board.input_decoder:
            return True      
    else:
        return False

def collect_players_and_suspects_list():
    """This function collects a list user inputs for players
       and suspects and decodes them.
       
       Args: none
       Returns: list of players decoded from valid user inputs
    """
    
    players_list = []
    while (players_input := input("Enter player: ")) != '#':
        i = players_input.upper()
        if not is_valid_player(i):
            print("Please enter a valid Suspect.")
            continue
        if i not in players_list:
            players_list.append(i)
    players_decoded = [Board.identify(player) for player in players_list]
    suspects_decoded = [Board.translate(player) for player in players_list]
    return players_decoded, suspects_decoded

def collect_players_list():
    """This function collects a list user inputs for players and decodes them.
       
       Args: none
       Returns: list of players decoded from valid user inputs
    """
    
    players_list = []
    while (players_input := input("Enter player: ")) != '#':
        i = players_input.upper()
        if not is_valid_player(i):
            print("Please enter a valid Suspect.")
            continue
        if i not in players_list:
            players_list.append(i)
    players_decoded = [Board.identify(player) for player in players_list]
    suspects_decoded = [Board.translate(player) for player in players_list]
    return players_decoded

def collect_cards():
    """This function collects a list user inputs for cards and decodes them.
       
       Args: none
       Returns: list of cards decoded from valid user inputs
    """
    
    cards_list = []
    while (cards_input := input("Enter card: ")) != '#':
        i = cards_input.upper()
        if not is_valid(i):
            print(f"Please enter a valid card.")
            continue
        cards_list.append(i)
    cards_decoded = [Board.translate(card) for card in cards_list]
    return cards_decoded

Overwriting errors.py
